In [3]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from keras.layers.convolutional import Conv2D

In [4]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [5]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [6]:
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/crawl-300d-2M-subword.vec'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))


earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)


Found 3437 unique tokens
../../dimensionalSM/data/crawl-300d-2M-subword.vec
G Word embeddings: 2000000
G Null word embeddings: 133


In [7]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [8]:
# split train/test
split = 8000

In [9]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [10]:
# API model
def model():
    # speech network   
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(input_mocap)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    # combined model
    model_combined = concatenate([ model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([ input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [11]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_train_text[:8000],x_train_mocap[:8000]],
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_train_text[8000:],x_train_mocap[8000:]], vad[8000:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('tm_fasttext_8000.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 60s 9ms/step - loss: 2.6463 - v_loss: 0.8979 - a_loss: 0.8750 - d_loss: 0.8734 - v_ccc: 0.1021 - a_ccc: 0.1250 - d_ccc: 0.1266 - val_loss: 2.2879 - val_v_loss: 0.8221 - val_a_loss: 0.7336 - val_d_loss: 0.7322 - val_v_ccc: 0.1779 - val_a_ccc: 0.2664 - val_d_ccc: 0.2678
Epoch 2/50
6400/6400 [==============================] - 55s 9ms/step - loss: 1.5641 - v_loss: 0.4365 - a_loss: 0.5691 - d_loss: 0.5585 - v_ccc: 0.5635 - a_ccc: 0.4309 - d_ccc: 0.4415 - val_loss: 1.9946 - val_v_loss: 0.7123 - val_a_loss: 0.6422 - val_d_loss: 0.6400 - val_v_ccc: 0.2877 - val_a_ccc: 0.3578 - val_d_ccc: 0.3600
Epoch 3/50
6400/6400 [==============================] - 54s 8ms/step - loss: 1.2717 - v_loss: 0.3155 - a_loss: 0.4823 - d_loss: 0.4739 - v_ccc: 0.6845 - a_ccc: 0.5177 - d_ccc: 0.5261 - val_loss: 2.0563 - val_v_loss: 0.76

Epoch 7/50
6400/6400 [==============================] - 57s 9ms/step - loss: 0.8986 - v_loss: 0.2101 - a_loss: 0.3516 - d_loss: 0.3369 - v_ccc: 0.7899 - a_ccc: 0.6484 - d_ccc: 0.6631 - val_loss: 1.9876 - val_v_loss: 0.6436 - val_a_loss: 0.6419 - val_d_loss: 0.7020 - val_v_ccc: 0.3564 - val_a_ccc: 0.3581 - val_d_ccc: 0.2980
Epoch 8/50
6400/6400 [==============================] - 56s 9ms/step - loss: 0.8416 - v_loss: 0.1926 - a_loss: 0.3341 - d_loss: 0.3148 - v_ccc: 0.8074 - a_ccc: 0.6659 - d_ccc: 0.6852 - val_loss: 1.8836 - val_v_loss: 0.5709 - val_a_loss: 0.6225 - val_d_loss: 0.6902 - val_v_ccc: 0.4291 - val_a_ccc: 0.3775 - val_d_ccc: 0.3098
Epoch 9/50
6400/6400 [==============================] - 54s 8ms/step - loss: 0.7833 - v_loss: 0.1834 - a_loss: 0.3044 - d_loss: 0.2955 - v_ccc: 0.8166 - a_ccc: 0.6956 - d_ccc: 0.7045 - val_loss: 1.9428 - val_v_loss: 0.5843 - val_a_loss: 0.6462 - val_d_loss: 0.7122 - val_v_ccc: 0.4157 - val_a_ccc: 0.3538 - val_d_ccc: 0.2878
Epoch 10/50
6400/6400 [==

Epoch 16/50
6400/6400 [==============================] - 54s 8ms/step - loss: 0.5295 - v_loss: 0.1244 - a_loss: 0.2081 - d_loss: 0.1970 - v_ccc: 0.8756 - a_ccc: 0.7919 - d_ccc: 0.8030 - val_loss: 1.9581 - val_v_loss: 0.5614 - val_a_loss: 0.6824 - val_d_loss: 0.7143 - val_v_ccc: 0.4386 - val_a_ccc: 0.3176 - val_d_ccc: 0.2857
Epoch 17/50
6400/6400 [==============================] - 53s 8ms/step - loss: 0.5098 - v_loss: 0.1222 - a_loss: 0.1989 - d_loss: 0.1886 - v_ccc: 0.8778 - a_ccc: 0.8011 - d_ccc: 0.8114 - val_loss: 1.9538 - val_v_loss: 0.5663 - val_a_loss: 0.6740 - val_d_loss: 0.7135 - val_v_ccc: 0.4337 - val_a_ccc: 0.3260 - val_d_ccc: 0.2865
Epoch 18/50
3808/6400 [================>.............] - ETA: 19s - loss: 0.4568 - v_loss: 0.1074 - a_loss: 0.1813 - d_loss: 0.1681 - v_ccc: 0.8926 - a_ccc: 0.8187 - d_ccc: 0.8319

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 53s 8ms/step - loss: 1.0334 - v_loss: 0.2492 - a_loss: 0.4021 - d_loss: 0.3821 - v_ccc: 0.7508 - a_ccc: 0.5979 - d_ccc: 0.6179 - val_loss: 1.8723 - val_v_loss: 0.5967 - val_a_loss: 0.6230 - val_d_loss: 0.6526 - val_v_ccc: 0.4033 - val_a_ccc: 0.3770 - val_d_ccc: 0.3474
Epoch 6/50
6400/6400 [==============================] - 53s 8ms/step - loss: 0.9542 - v_loss: 0.2257 - a_loss: 0.3769 - d_loss: 0.3516 - v_ccc: 0.7743 - a_ccc: 0.6231 - d_ccc: 0.6484 - val_loss: 1.8607 - val_v_loss: 0.5945 - val_a_loss: 0.6088 - val_d_loss: 0.6574 - val_v_ccc: 0.4055 - val_a_ccc: 0.3912 - val_d_ccc: 0.3426
Epoch 7/50
6400/6400 [==============================] - 56s 9ms/step - loss: 0.8915 - v_loss: 0.2101 - a_loss: 0.3492 - d_loss: 0.3323 - v_ccc: 0.7899 - a_ccc: 0.6508 - d_ccc: 0.6677 - val_loss: 1.8696 - val_v_loss: 0.5797 - val_a_loss: 0.6217 - val_d_loss: 0.6682 - val_v_ccc: 0.4203 - val_a_ccc: 0.3783 - val_d_ccc: 0.3318
Epoch 8/50
6400/6400 [==============

...

6400/6400 [==============================] - 59s 9ms/step - loss: 0.8227 - v_loss: 0.1947 - a_loss: 0.3151 - d_loss: 0.3129 - v_ccc: 0.8053 - a_ccc: 0.6849 - d_ccc: 0.6871 - val_loss: 1.8774 - val_v_loss: 0.5731 - val_a_loss: 0.6200 - val_d_loss: 0.6842 - val_v_ccc: 0.4269 - val_a_ccc: 0.3800 - val_d_ccc: 0.3158
Epoch 10/50
6400/6400 [==============================] - 59s 9ms/step - loss: 0.7731 - v_loss: 0.1806 - a_loss: 0.3025 - d_loss: 0.2900 - v_ccc: 0.8194 - a_ccc: 0.6975 - d_ccc: 0.7100 - val_loss: 1.8817 - val_v_loss: 0.5756 - val_a_loss: 0.6293 - val_d_loss: 0.6769 - val_v_ccc: 0.4244 - val_a_ccc: 0.3707 - val_d_ccc: 0.3231
Epoch 11/50
6400/6400 [==============================] - 59s 9ms/step - loss: 0.7218 - v_loss: 0.1690 - a_loss: 0.2774 - d_loss: 0.2753 - v_ccc: 0.8310 - a_ccc: 0.7226 - d_ccc: 0.7247 - val_loss: 1.8872 - val_v_loss: 0.5772 - val_a_loss: 0.6375 - val_d_loss: 0.6725 - val_v_ccc: 0.4228 - val_a_ccc: 0.3625 - val_d_ccc: 0.3275
Epoch 12/50
6400/6400 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 58s 9ms/step - loss: 0.7872 - v_loss: 0.1833 - a_loss: 0.3054 - d_loss: 0.2985 - v_ccc: 0.8167 - a_ccc: 0.6946 - d_ccc: 0.7015 - val_loss: 1.9031 - val_v_loss: 0.5824 - val_a_loss: 0.6382 - val_d_loss: 0.6824 - val_v_ccc: 0.4176 - val_a_ccc: 0.3618 - val_d_ccc: 0.3176
Epoch 11/50
6400/6400 [==============================] - 57s 9ms/step - loss: 0.7428 - v_loss: 0.1760 - a_loss: 0.2864 - d_loss: 0.2805 - v_ccc: 0.8240 - a_ccc: 0.7136 - d_ccc: 0.7195 - val_loss: 1.8963 - val_v_loss: 0.5661 - val_a_loss: 0.6489 - val_d_loss: 0.6814 - val_v_ccc: 0.4339 - val_a_ccc: 0.3511 - val_d_ccc: 0.3186
Epoch 12/50
6400/6400 [==============================] - 58s 9ms/step - loss: 0.6901 - v_loss: 0.1658 - a_loss: 0.2643 - d_loss: 0.2599 - v_ccc: 0.8342 - a_ccc: 0.7357 - d_ccc: 0.7401 - val_loss: 1.9246 - val_v_loss: 0.6019 - val_a_loss: 0.6392 - val_d_loss: 0.6835 - val_v_ccc: 0.3981 - val_a_ccc: 0.3608 - val_d_ccc: 0.3165
Epoch 13/50
6400/6400 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 59s 9ms/step - loss: 0.7133 - v_loss: 0.1659 - a_loss: 0.2760 - d_loss: 0.2714 - v_ccc: 0.8341 - a_ccc: 0.7240 - d_ccc: 0.7286 - val_loss: 1.9263 - val_v_loss: 0.5622 - val_a_loss: 0.6530 - val_d_loss: 0.7111 - val_v_ccc: 0.4378 - val_a_ccc: 0.3470 - val_d_ccc: 0.2889
Epoch 12/50
6400/6400 [==============================] - 58s 9ms/step - loss: 0.6707 - v_loss: 0.1610 - a_loss: 0.2590 - d_loss: 0.2508 - v_ccc: 0.8390 - a_ccc: 0.7410 - d_ccc: 0.7492 - val_loss: 1.8949 - val_v_loss: 0.5614 - val_a_loss: 0.6290 - val_d_loss: 0.7045 - val_v_ccc: 0.4386 - val_a_ccc: 0.3710 - val_d_ccc: 0.2955
Epoch 13/50
6400/6400 [==============================] - 59s 9ms/step - loss: 0.6294 - v_loss: 0.1504 - a_loss: 0.2428 - d_loss: 0.2362 - v_ccc: 0.8496 - a_ccc: 0.7572 - d_ccc: 0.7638 - val_loss: 1.9156 - val_v_loss: 0.5803 - val_a_loss: 0.6488 - val_d_loss: 0.6865 - val_v_ccc: 0.4197 - val_a_ccc: 0.3512 - val_d_ccc: 0.3135
Epoch 14/50
6400/6400 [===========